In [79]:
# src code is available @https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb
# this is just a toy DataSet for the demo
training_data = [['Green',3,'Apple'],
                ['Yellow',3,'Apple'],
                ['Red',1,'Grape'],
                ['Red',1,'Grape'],
                ['Yellow',3,'Lemon']]
print (training_data)

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape'], ['Yellow', 3, 'Lemon']]


In [48]:
# headers
headers = ['color','diameter','label']
print (headers)

['color', 'diameter', 'label']


In [6]:
def unique_vals(rows, col):
    """Find the unique values for a column in dataset"""
    return set([row[col] for row in rows])

In [10]:
# testing
unique_vals(training_data,0)
# unique_vals(training_data,1)

{'Green', 'Red', 'Yellow'}

In [42]:
def class_counts(rows):
    """Counts number of each type in the dataset"""
    counts = {} # a dictionary of label -> count.
    for row in rows:
        """label is last column in our dataset so, row[-1] is equal to row[2]"""
        label = row[-1]
#         print ('now the label is : %s'%label)
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# testing
print (class_counts(training_data))

{'Lemon': 1, 'Grape': 2, 'Apple': 2}


In [44]:
def isnumeric(value):
    """Checks if the value is numeric or not"""
    return isinstance(value, int) or isinstance(value, float)

# testing
print(isnumeric('4'))

False


In [ ]:
class Question:
    """A Question is used to partition a dataset.
    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """
    def __init__(self,column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        """compares the feature value in an example to the feature value in the question"""
        val = example[self.column]
        if isnumeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        """this is just a helper method to print question in readable format"""
        condition = "=="
        if isnumeric(self.value):
            condition = ">="
        return "Is %s %s %s" % (headers[self.column],condition,str(self.value))
    
# testing
Question(0,'Yellow')

Is color == Yellow

In [55]:
Question(2,'Apple')

Is label == Apple

In [58]:
Question(1,6)

Is diameter >= 6

In [67]:
q = Question(0, 'Green')
q

Is color == Green

In [62]:
example = training_data[0]
example

['Green', 3, 'Apple']

In [68]:
q.match(example)

True

In [72]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'."""
    true_rows, false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
            
        else:
            false_rows.append(row)
    return true_rows, false_rows

# testing
true_rows, false_rows = partition(training_data,q)
true_rows

[['Green', 3, 'Apple']]

In [73]:
false_rows

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [75]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label]/float(len(rows))
        impurity -= prob_of_label ** 2
    return impurity

In [76]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [77]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [82]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.7999999999999998

In [ ]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [83]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999